In [1]:
from ctu_crs import CRS97
import numpy as np
from numpy.typing import NDArray
import time
import json
import cv2
import PoseComposer
import camera
import puzzle

In [2]:
robot = CRS97()

Firmware version : MARS8 v1.0 build Pi Oct  2 2017 11:06:45


In [3]:
robot.initialize(home=False)

In [4]:
robot.soft_home()

In [17]:
robot.release()

In [5]:
def move_robot_to(position):
    q0 = robot.q_home
    ik_sols = robot.ik(position)
    assert len(ik_sols) > 0
    closest_solution = min(ik_sols, key=lambda q: np.linalg.norm(q - q0))
    robot.move_to_q(closest_solution)
    robot.wait_for_motion_stop()

In [31]:
cam = camera.Camera()
img = robot.grab_image()
midpoint, R = cam.get_mid_points(img)
centerInRobotCoords = cam.cameraToRobot(midpoint)

[[2]
 [1]]


In [32]:
midpoint

array([  32.92514861,   86.35585775, 1174.49572153])

In [33]:
R

array([[-0.32006094, -0.94739696,  0.        ],
       [ 0.94739696, -0.32006094,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [34]:
centerInRobotCoords

array([ 0.50931872, -0.01911265,  0.01829823])

In [35]:
maze = puzzle.PuzzleC()
trajectory_R = maze.get_reverse_trajectory()
trajectory_F  = trajectory_R[::-1]
trajectory_F[:, 2] *= -1
trajectory_F

array([[ -50.        ,   50.        , -200.        ],
       [ -50.        ,   50.        , -194.44444444],
       [ -50.        ,   50.        , -188.88888889],
       [ -50.        ,   50.        , -183.33333333],
       [ -50.        ,   50.        , -177.77777778],
       [ -50.        ,   50.        , -172.22222222],
       [ -50.        ,   50.        , -166.66666667],
       [ -50.        ,   50.        , -161.11111111],
       [ -50.        ,   50.        , -155.55555556],
       [ -50.        ,   50.        , -150.        ],
       [ -50.        ,   50.        , -150.        ],
       [ -50.        ,   44.44444444, -144.44444444],
       [ -50.        ,   38.88888889, -138.88888889],
       [ -50.        ,   33.33333333, -133.33333333],
       [ -50.        ,   27.77777778, -127.77777778],
       [ -50.        ,   22.22222222, -122.22222222],
       [ -50.        ,   16.66666667, -116.66666667],
       [ -50.        ,   11.11111111, -111.11111111],
       [ -50.        ,    5.

In [36]:
trajectory_F_camera_coord = trajectory_F @ R + midpoint
trajectory_F_camera_coord

array([[  96.29804362,  117.72265834,  974.49572153],
       [  96.29804362,  117.72265834,  980.05127708],
       [  96.29804362,  117.72265834,  985.60683264],
       [  96.29804362,  117.72265834,  991.1623882 ],
       [  96.29804362,  117.72265834,  996.71794375],
       [  96.29804362,  117.72265834, 1002.27349931],
       [  96.29804362,  117.72265834, 1007.82905486],
       [  96.29804362,  117.72265834, 1013.38461042],
       [  96.29804362,  117.72265834, 1018.94016597],
       [  96.29804362,  117.72265834, 1024.49572153],
       [  96.29804362,  117.72265834, 1024.49572153],
       [  91.0347272 ,  119.50077469, 1030.05127708],
       [  85.77141077,  121.27889105, 1035.60683264],
       [  80.50809435,  123.05700741, 1041.1623882 ],
       [  75.24477793,  124.83512376, 1046.71794375],
       [  69.98146151,  126.61324012, 1052.27349931],
       [  64.71814509,  128.39135648, 1057.82905486],
       [  59.45482867,  130.16947283, 1063.38461042],
       [  54.19151225,  131.

In [37]:
trajectory_F_robot_coord = [cam.cameraToRobot(x) for x in trajectory_F_camera_coord]    
# for p in trajectory_F_robot_coord:
#     p[2] += 0.05
trajectory_F_robot_coord

[array([0.55179538, 0.05709891, 0.21165679]),
 array([0.55147667, 0.05673964, 0.20612203]),
 array([0.55115797, 0.05638036, 0.20058727]),
 array([0.55083927, 0.05602109, 0.19505251]),
 array([0.55052057, 0.05566182, 0.18951775]),
 array([0.55020186, 0.05530254, 0.18398299]),
 array([0.54988316, 0.05494327, 0.17844824]),
 array([0.54956446, 0.05458399, 0.17291348]),
 array([0.54924575, 0.05422472, 0.16737872]),
 array([0.54892705, 0.05386545, 0.16184396]),
 array([0.54892705, 0.05386545, 0.16184396]),
 array([0.55040938, 0.04825601, 0.1565463 ]),
 array([0.5518917 , 0.04264656, 0.15124863]),
 array([0.55337403, 0.03703712, 0.14595097]),
 array([0.55485636, 0.03142768, 0.1406533 ]),
 array([0.55633869, 0.02581824, 0.13535564]),
 array([0.55782102, 0.02020879, 0.13005797]),
 array([0.55930334, 0.01459935, 0.12476031]),
 array([0.56078567, 0.00898991, 0.11946264]),
 array([0.562268  , 0.00338047, 0.11416498]),
 array([0.562268  , 0.00338047, 0.11416498]),
 array([0.55670345, 0.0012405 , 0.

In [38]:
start = trajectory_F_robot_coord[0]
start[2] = start[2] + 0.1
start

array([0.55179538, 0.05709891, 0.31165679])

In [39]:
arm_vector = np.array([-1, 1, 0])
normal_vector = np.array([0, 0, 1])
StartPose = PoseComposer.make_se3_matrix(start, arm_vector, normal_vector)
StartPose

array([[-0.70710678,  0.70710678,  0.        ,  0.45633596],
       [ 0.70710678,  0.70710678,  0.        ,  0.15255833],
       [ 0.        ,  0.        , -1.        ,  0.31165679],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [40]:
move_robot_to(StartPose)

In [41]:
for position in trajectory_F_robot_coord:
    Pose = PoseComposer.make_se3_matrix(position, arm_vector, normal_vector)
    move_robot_to(Pose)

Exception: Check ready: error, arm power is off, motion stop.

In [17]:
for position in trajectory_F_robot_coord[::-1]:
    Pose = PoseComposer.make_se3_matrix(position, arm_vector, normal_vector)
    move_robot_to(Pose)

In [18]:
import solution

In [19]:
maze = puzzle.PuzzleC()
solution.solution(maze, robot)

[[2]
 [1]]


In [44]:
robot.release()


In [43]:
robot.soft_home()

In [42]:
robot.initialize(home=False)